### Step 1: Load Data and Extract Frames

In [20]:
import cv2
import pandas as pd
import os

data_dir = 'datasets/Subject_1'
output_dir = 'outputs/Subject_1'

def extract_frames(video_path, intervals, save_dir, action_type):
    action_folder = os.path.join(save_dir, action_type)  # Path to action-specific folder
    if not os.path.exists(action_folder):
        os.makedirs(action_folder)
    
    video_name = os.path.basename(video_path).split('.')[0]
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = 0
    
    for interval in intervals:
        action, start, end = interval['action'], interval['start'], interval['end']
        cap.set(cv2.CAP_PROP_POS_FRAMES, start * fps)
        
        for i in range(int(start * fps), int(end * fps)):
            ret, frame = cap.read()
            if not ret:
                break
            # Save frame in the appropriate action folder
            frame_filename = f"{video_name}_{action}_{frame_count}.jpg"
            frame_path = os.path.join(action_folder, frame_filename)
            cv2.imwrite(frame_path, frame)
            frame_count += 1
    cap.release()

def load_csv_and_extract(csv_path, video_folder, action_type):
    data = pd.read_csv(csv_path)
    for idx, row in data.iterrows():
        intervals = []
        for activity in row['Classes'].split(';'):
            action, time_range = activity.split('[')
            start, end = map(float, time_range.strip(']').split(' to '))
            intervals.append({'action': action.strip(), 'start': start, 'end': end})
        
        video_path = os.path.join(video_folder, row['File Name'])
        extract_frames(video_path, intervals, output_dir, action_type)

# Example usage
load_csv_and_extract('datasets/Subject_1/ADL.csv', os.path.join(data_dir, 'ADL'), 'ADL')
load_csv_and_extract('datasets/Subject_1/Fall.csv', os.path.join(data_dir, 'Fall'), 'Fall')


### Step 2: Pose Detection with PyTorch

In [21]:
import os
import numpy as np
import torch
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Configure Detectron2 for pose estimation
cfg = get_cfg()
cfg.MODEL.DEVICE = "cpu"  # Use "cuda" if you have a GPU with CUDA support
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

# Function to detect keypoints in an image
def detect_pose(image_path):
    image = cv2.imread(image_path)
    outputs = predictor(image)
    keypoints = outputs["instances"].pred_keypoints.to("cpu").numpy()
    return keypoints

# Function to flatten keypoints for classifier input
def flatten_keypoints(keypoints):
    flattened = keypoints[:, :2].flatten()  # Use only x, y coordinates
    return flattened

# Directory for frames
frame_dir = 'outputs/Subject_1'
keypoints_data = []
labels = []

# Process frames in both ADL and Fall folders
for label_name in ['ADL', 'Fall']:
    label_dir = os.path.join(frame_dir, label_name)
    label = 1 if label_name == 'Fall' else 0  # 1 for Fall, 0 for ADL
    
    for filename in os.listdir(label_dir):
        if filename.endswith(".jpg"):
            frame_path = os.path.join(label_dir, filename)
            
            # Detect keypoints in frame
            keypoints = detect_pose(frame_path)
            if len(keypoints) > 0:  # Ensure keypoints were detected
                flattened_keypoints = flatten_keypoints(keypoints[0])  # [0] for first person detected
                keypoints_data.append(flattened_keypoints)
                
                # Append label
                labels.append(label)

# Convert to arrays for classifier training
keypoints_data = np.array(keypoints_data)
labels = np.array(labels)

print("Keypoints data shape:", keypoints_data.shape)
print("Labels shape:", labels.shape)


KeyboardInterrupt: 

### Step 3: Train a Fall Detection Classifier with PyTorch

### Step 4: Use Pre-trained Model for Sleeping Falls